<a href="https://colab.research.google.com/github/gustavosr8/SpiroPhone/blob/master/Espirometria_regressionN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import librosa
import librosa.display

import scipy

from scipy import signal
from scipy.signal import butter, filtfilt

import sklearn
from sklearn import preprocessing, decomposition, model_selection, ensemble, metrics, pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score, accuracy_score, explained_variance_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Activation

!pip install scikeras
from scikeras.wrappers import KerasRegressor


#Leitura dos dados



In [ ]:
drive.mount('/content/drive')
path='/content/drive/MyDrive/IC/codigo/assets/'
dataset = 'datasetn.csv'

df = pd.read_csv(path+dataset)
df.head()

# Checando se arquivos existem, numero de canais e taxa de amostragem
n_files = 0
missing_files = 0
taxas_de_amostragem = []
duracoes = []
amostras = []
classe = []
for index, row in df.iterrows():
  full_fname = path + str(row['REGISTRO'])+'.wav'
  if (n_files % 10 == 0):
    print(str(n_files+1) + " " + full_fname)
  if not os.path.exists(full_fname):
    print(str(row['REGISTRO']) + " (index=" + str(n_files) + ") does not exist!")
    df.drop(df.index[n_files])
    missing_files += 1
    n_files += 1
  else:
    y, sr = librosa.load(full_fname, sr=None, mono=True)
    taxas_de_amostragem.append(sr)
    duracoes.append(len(y)/sr)
    amostras.append(y)
    n_files += 1
print("Encontrei esse numero de arquivos: " + str(n_files-missing_files))
df['sr'] = taxas_de_amostragem
df['len'] = duracoes

Mounted at /content/drive
1 /content/drive/MyDrive/IC/codigo/assets/4306486.wav
11 /content/drive/MyDrive/IC/codigo/assets/10348393.wav
21 /content/drive/MyDrive/IC/codigo/assets/4340095.wav
31 /content/drive/MyDrive/IC/codigo/assets/10305974n.wav
41 /content/drive/MyDrive/IC/codigo/assets/863418n.wav
Encontrei esse numero de arquivos: 46


In [ ]:
teste1, sr = librosa.load(path+'10509684.wav', sr=None, mono=True)
teste2, sr = librosa.load(path+'10523145.wav', sr=None, mono=True)

In [ ]:
df

,REGISTRO,SEXO,IDADE,PESO,ALTURA,CVF,CVF%,VEF1,VEF1%,VEF1/CVF,VEF1/CVF%,FEF,FEF%,PF,PF%,DISTÚRBIO,sr,len
0,4306486,F,71,76.9,160.0,2.58,92,1.63,76,0.63,81,1.17,57,4.30,74,OBSTRUÇÃO,48000,11.677125
1,10429556,M,63,91.6,187.5,4.33,85,2.52,63,0.58,74,2.05,58,7.21,72,OBSTRUÇÃO,48000,12.490104
2,3724996,F,65,58.5,162.0,2.50,83,1.77,75,0.71,90,1.21,55,4.84,80,OBSTRUÇÃO,48000,17.440042
3,10433358,F,45,61.6,146.0,2.12,80,1.16,52,0.54,66,0.50,20,2.91,50,OBSTRUÇÃO,48000,14.128250
4,10425372,F,56,65.8,149.0,1.48,55,0.58,26,0.39,47,0.19,8,2.02,31,OBSTRUÇÃO,48000,14.616604
5,4481868,F,69,81.1,174.0,3.41,82,2.57,81,0.75,97,1.97,69,8.09,95,NORMAL,48000,9.181437
6,10082908,F,39,66.9,153.0,3.25,107,2.46,95,0.75,89,1.95,68,6.72,107,OBSTRUÇÃO,48000,12.798333
7,10305974,M,70,71.0,172.0,2.98,74,1.66,54,0.56,72,0.58,20,4.77,57,OBSTRUÇÃO,48000,13.218063
8,4432769,F,60,61.4,146.0,1.87,78,1.48,78,0.79,99,1.48,73,4.54,85,RESTRIÇÃO,48000,12.182771
9,522460,F,70,74.4,151.0,2.05,84,1.58,85,0.77,98,1.45,76,5.18,98,NORMAL,48000,11.457896


#Processamento de Features

##Pressão e fluxo de ar aproximados

A pressão proporcional de ar de saída é dada por uma função de transferência do áudio original.

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / (0.5*fs)
    # Pega os coeficientes do filtro
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def butter_highpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / (0.5*fs)
    # Pega os coeficientes do filtro
    b, a = butter(order, normal_cutoff, btype='highpass', analog=False)
    y = filtfilt(b, a, data)
    return y

# Requerimentos do filtro
fs = sr         # sample rate, Hz
cutoff = 1000      # Frequência de corte do filtro, em Hz
order = 5       # Ordem do filtro

plips=[]
for i in amostras:
  plips.append(butter_highpass_filter(i, cutoff, fs, order))

rlips=  0.05                                    #raio de abertura da boca de 5cm
ulips=[]

for p in plips:
  ulips.append(2*np.pi*rlips**2*np.sqrt(2*abs(p)))   #proporcional ao fluxo de ar saindo da boca

amplitude_envelope_plips = []
amplitude_envelope_ulips = []

for i in range(len(plips)):
  amplitude_envelope_plips.append(librosa.feature.rms(y=plips[i])[0,:])
  amplitude_envelope_ulips.append(librosa.feature.rms(y=ulips[i])[0,:])

In [ ]:
def descritores(ulips, plips, df_row ):
  return np.array([np.max(ulips), np.sum(ulips), np.sum(ulips[0:df_row['sr']]), df_row['IDADE'], df_row['PESO'], df_row['ALTURA']])

n_files = 0
vetores = []
for i in range(len(ulips)):
  vetores.append(descritores(amplitude_envelope_ulips[i], amplitude_envelope_plips[i], df.iloc[i]))
  n_files +=1
print("Processei esse número de arquivos: " + str(n_files))
vetores = np.array(vetores)
print(vetores.shape)

scaler = StandardScaler()
scaler.fit(vetores)
dados = scaler.transform(vetores)
params = ['CVF', 'VEF1','PF']


Processei esse número de arquivos: 46
(46, 6)


In [ ]:
plips_test=[butter_highpass_filter(teste1, cutoff, fs, order),
            butter_highpass_filter(teste2, cutoff, fs, order)]

ulips_test=[]
for p in plips_test:
  ulips_test.append(2*np.pi*rlips**2*np.sqrt(2*abs(p)))   #proporcional ao fluxo de ar saindo da boca

amplitude_envelope_plips_test = []
amplitude_envelope_ulips_test = []

for i in range(len(plips_test)):
  amplitude_envelope_plips_test.append(librosa.feature.rms(y=plips_test[i])[0,:])
  amplitude_envelope_ulips_test.append(librosa.feature.rms(y=ulips_test[i])[0,:])

In [ ]:
vetores_test = [np.array([np.max(amplitude_envelope_ulips[0]), np.sum(amplitude_envelope_ulips[0]), np.sum(amplitude_envelope_ulips[0][0:48000]), 55, 78.6, 158]),
                np.array([np.max(amplitude_envelope_ulips[1]), np.sum(amplitude_envelope_ulips[1]), np.sum(amplitude_envelope_ulips[1][0:48000]), 66, 78.6, 171])]
scaler = StandardScaler()
scaler.fit(vetores_test)
dados_test = scaler.transform(vetores_test)

#Random forest regressor

In [ ]:
def RandomForest(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Fitting Random Forest Regression to the dataset

  # create regressor object
  regressor = RandomForestRegressor(n_estimators=1000, random_state=42, verbose=0, n_jobs=-1, oob_score=True)

  scores = cross_val_score(regressor, X, y, cv=5, scoring='explained_variance')
  # fit the regressor with x and y data
  regressor.fit(X, y)

  y_pred = regressor.predict(dados_test)
  print(param)
  print("predicted:")
  print(y_pred)
  #print("goal:")
  #print(y_test)
  #print("score:")
  #print(explained_variance_score(y_test, y_pred))
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")

for param in params:
  RandomForest(param)

CVF
predicted:
[2.45405 3.1606 ]
0.85 accuracy with a standard deviation of 0.07
.
.
.
VEF1
predicted:
[2.30971 2.392  ]
0.83 accuracy with a standard deviation of 0.08
.
.
.
PF
predicted:
[6.86392 7.35353]
0.82 accuracy with a standard deviation of 0.06
.
.
.


#Regressor linear


In [ ]:
def RegressorLinear(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Fitting Random Forest Regression to the dataset

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  # create regressor object
  model = Sequential()
  # Define the model consisting of a single neuron.
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)


  scores = []
  #scores = cross_val_score(model, X, y, cv=5, scoring='explained_variance')
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X[train], y[train], epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))
    #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    #loss_per_fold.append(scores[0])
    #fold_no += 1
  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test, verbose = 0)


  #y_pred_=[]
  #for i in y_pred:
  #  y_pred_.append(i[0])
  #y_pred = y_pred_
  print(param)
  print("predicted:")
  print(y_pred)
  #print("goal:")
  #print(y_test)
  #print("score:")
  #print(explained_variance_score(y_test,y_pred))
  #print(scores)
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  RegressorLinear(param)

CVF
predicted:
[[2.9122713]
 [2.9454024]]
0.68 accuracy with a standard deviation of 0.21
.
.
.
VEF1
predicted:
[[2.3023515]
 [1.6954584]]
0.46 accuracy with a standard deviation of 0.17
.
.
.
PF
predicted:
[[6.4953814]
 [4.8460846]]
0.24 accuracy with a standard deviation of 0.54
.
.
.


In [ ]:
def MLP(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Fitting Random Forest Regression to the dataset

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  model = Sequential()
  # create regressor object
  model.add(Dense(3, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
  model.add(Dense(6, kernel_initializer='normal'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)

  scores = []
  #scores = cross_val_score(model, X, y, cv=5, scoring='explained_variance')
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X_train, y_train, epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))
    #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    #loss_per_fold.append(scores[0])
    #fold_no += 1
  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test)

  print(param)
  print("predicted:")
  print(y_pred)
  #print("goal:")
  #print(y_test)
  #print("score:")
  #print(explained_variance_score(y_test,y_pred))
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  MLP(param)


1/1 [==============================] - 0s 38ms/step
CVF
predicted:
[[2.690206]
 [3.259498]]
0.90 accuracy with a standard deviation of 0.05
.
.
.
1/1 [==============================] - 0s 26ms/step
VEF1
predicted:
[[2.2280874]
 [1.7304606]]
0.69 accuracy with a standard deviation of 0.17
.
.
.
1/1 [==============================] - 0s 22ms/step
PF
predicted:
[[6.7346163]
 [5.222809 ]]
0.58 accuracy with a standard deviation of 0.22
.
.
.


In [ ]:
def MLP(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Fitting Random Forest Regression to the dataset

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  model = Sequential()
  # create regressor object
  model.add(Dense(6, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
  model.add(Dense(3, kernel_initializer='normal'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)

  scores = []
  #scores = cross_val_score(model, X, y, cv=5, scoring='explained_variance')
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X_train, y_train, epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))
    #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    #loss_per_fold.append(scores[0])
    #fold_no += 1
  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test)

  print(param)
  print("predicted:")
  print(y_pred)
  #print("goal:")
  #print(y_test)
  #print("score:")
  #print(explained_variance_score(y_test,y_pred))
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  MLP(param)


1/1 [==============================] - 0s 23ms/step
CVF
predicted:
[[2.2880404]
 [3.2133238]]
0.80 accuracy with a standard deviation of 0.09
.
.
.
1/1 [==============================] - 0s 22ms/step
VEF1
predicted:
[[2.0019526]
 [1.6716396]]
0.68 accuracy with a standard deviation of 0.16
.
.
.
1/1 [==============================] - 0s 34ms/step
PF
predicted:
[[6.273751]
 [5.085983]]
0.70 accuracy with a standard deviation of 0.20
.
.
.


In [ ]:
def MLP(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  # Fitting Random Forest Regression to the dataset

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  model = Sequential()
  # create regressor object
  model.add(Dense(10, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
  model.add(Dense(8, kernel_initializer='normal'))
  model.add(Dense(6, kernel_initializer='normal'))
  model.add(Dense(4, kernel_initializer='normal'))
  model.add(Dense(2, kernel_initializer='normal'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)

  scores = []
  #scores = cross_val_score(model, X, y, cv=5, scoring='explained_variance')
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X_train, y_train, epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))
    #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    #loss_per_fold.append(scores[0])
    #fold_no += 1
  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test)

  print(param)
  print("predicted:")
  print(y_pred)
  #print("goal:")
  #print(y_test)
  #print("score:")
  #print(explained_variance_score(y_test,y_pred))
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  MLP(param)


1/1 [==============================] - 0s 30ms/step
CVF
predicted:
[[2.6319528]
 [3.1102068]]
0.85 accuracy with a standard deviation of 0.07
.
.
.
1/1 [==============================] - 0s 36ms/step
VEF1
predicted:
[[1.989645 ]
 [1.8057649]]
0.73 accuracy with a standard deviation of 0.17
.
.
.
1/1 [==============================] - 0s 22ms/step
PF
predicted:
[[6.457251 ]
 [5.1127567]]
0.66 accuracy with a standard deviation of 0.22
.
.
.
